In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from scipy.stats import uniform, randint
from xgboost import XGBRegressor
from paramFunctions import *

import warnings

warnings.filterwarnings("ignore")

In [2]:
df_phone = pd.read_csv("../data/Cellphone.csv")

In [3]:
df_phone = pd.read_csv("../data/Cellphone.csv")
X_phone = df_phone.drop(columns=["Price", "Product_id"], axis=1)
Y_phone = df_phone['Price']

X_phone_train, X_phone_test, y_phone_train, y_phone_test = train_test_split(X_phone, Y_phone, test_size=0.3, random_state=42)

In [4]:
si_cat=SimpleImputer(strategy="constant", fill_value='NO INFORMATION')
si_num = SimpleImputer(strategy='median')
scaler=MinMaxScaler()

numerical_pipeline = Pipeline(steps=[
    ('imputer', si_num),
    ('scaler', scaler)
    ])
    
category_pipeline = Pipeline(steps=[
    ('imputer', si_cat),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
    ])

col_transformer = ColumnTransformer(transformers=[
    ('num', numerical_pipeline, make_column_selector(dtype_include=np.number)),
    ('cat', category_pipeline, make_column_selector(dtype_include='object'))
    ],
    remainder='drop',
    n_jobs=-1)  

In [5]:
RF_regressor = Pipeline([('preprocessor', col_transformer),
                         ('model', RandomForestRegressor())])
ElasticNet_regressor = Pipeline([('preprocessor', col_transformer),
                                 ('model', ElasticNet())])    
XGB_regressor = Pipeline([('preprocessor', col_transformer),
                          ('model', XGBRegressor())])  

In [9]:
param_distributions_rf = {
    'model__n_estimators': randint(1,500),
    'model__min_samples_split': randint(2, 10),
    'model__min_samples_leaf': randint(2, 10)
}

param_distributions_enet = {
    'model__alpha': uniform(0, 1),
    'model__l1_ratio': uniform(0, 1),
}

param_distributions_xgb = {
    'model__min_child_weight': [2 ** uniform.rvs(0, 7) for _ in range(100)],
    'model__max_depth': randint(1, 15),
    'model__colsample_bytree': uniform(0, 1)
}

In [30]:
param_list_enet, score_list_enet = get_param_score_list(X_phone_train,y_phone_train,X_phone_test, y_phone_test, ElasticNet_regressor, param_distributions_enet, 160)

In [25]:
param_list_xgb, score_list_xgb = get_param_score_list(X_phone_train,y_phone_train,X_phone_test, y_phone_test, XGB_regressor, param_distributions_enet, 160)

In [10]:
param_list_rf, score_list_rf = get_param_score_list(X_phone_train,y_phone_train,X_phone_test, y_phone_test, RF_regressor, param_distributions_rf, 160)

In [13]:
list_1 = list(range(2, 12, 2))  
list_2 = list(range(20, 151, 10))  

iter_number = list_1 + list_2

In [14]:
r2_iteration_plot(iter_number, score_list_rf,'phone', 'rf')

In [31]:
r2_iteration_plot(iter_number, score_list_enet,'phone', 'enet')

In [32]:
r2_iteration_plot(iter_number, score_list_xgb,'phone', 'xgb')

In [16]:
r2_all_models(iter_number, score_list_rf,score_list_xgb, score_list_enet, 'phone')